# Advanced Topics: Additional Filtering

The filtering examples we've shown to this point have been pretty simple, either comparisons between columns or fixed values, or set filter functions like `isin` and `notin`. 

Ibis supports a number of richer analytical filters that can involve one or more of:

- Aggregates computed from the same or other tables
- Conditional aggregates (in SQL-speak these are similar to "correlated subqueries")
- "Existence" set filters (equivalent to the SQL `EXISTS` and `NOT EXISTS` keywords)

## Setup

In [ ]:
import os
import ibis

ibis.options.interactive = True

connection = ibis.sqlite.connect(os.path.join('data', 'geography.db'))

## Using scalar aggregates in filters

In [ ]:
countries = connection.table('countries')
countries.limit(5)

We could always compute some aggregate value from the table and use that in another expression, or we can use a data-derived aggregate in the filter. Take the average of a column. For example the average of countries size:

In [ ]:
countries.area_km2.mean()

You can use this expression as a substitute for a scalar value in a filter, and the execution engine will combine everything into a single query rather than having to access the database multiple times. For example, we want to filter European countries larger than the average country size in the world. See how most countries in Europe are smaller than the world average:

In [ ]:
cond = countries.area_km2 > countries.area_km2.mean()
expr = countries[(countries.continent == 'EU') & cond]
expr

## Conditional aggregates


Suppose that we wish to filter using an aggregate computed conditional on some other expressions holding true.

For example, we want to filter European countries larger than the average country size, but this time of the average in Africa. African countries have an smaller size compared to the world average, and France gets into the list:

In [ ]:
conditional_avg = countries[countries.continent == 'AF'].area_km2.mean()
countries[(countries.continent == 'EU') & (countries.area_km2 > conditional_avg)]

## "Existence" filters


Some filtering involves checking for the existence of a particular value in a column of another table, or amount the results of some value expression. This is common in many-to-many relationships, and can be performed in numerous different ways, but it's nice to be able to express it with a single concise statement and let Ibis compute it optimally.

An example could be finding all countries that had **any** year with a higher GDP than 3 trillion US dollars:

In [ ]:
gdp = connection.table('gdp')
gdp

In [ ]:
cond = ((gdp.country_code == countries.iso_alpha3) &
        (gdp.value > 3e12)).any()

countries[cond]['name']

Note how this is different than a join between `countries` and `gdp`, which would return one row per year. The method `.any()` is equivalent to filtering with a subquery.

## Filtering in aggregations


Suppose that you want to compute an aggregation with a subset of the data for _only one_ of the metrics / aggregates in question, and the complete data set with the other aggregates. Most aggregation functions are thus equipped with a `where` argument. Let me show it to you in action:

In [ ]:
arctic = countries.name.isin(['United States',
                              'Canada',
                              'Finland',
                              'Greenland',
                              'Iceland',
                              'Norway',
                              'Russia',
                              'Sweden'])

metrics = [countries.count().name('# countries'),
           countries.population.sum().name('total population'),
           countries.population.sum(where=arctic).name('population arctic countries')]

(countries.groupby(countries.continent)
          .aggregate(metrics))